In [1]:
import datafusion
from datafusion import column
from datafusion import functions as f

ctx = datafusion.SessionContext()
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet/indexed/file_id/*.parquet")
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet/indexed/file_id/*.parquet")
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet/*.parquet")
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet/*.parquet")
ctx

In [ ]:
df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet/*.parquet")
df

In [23]:
%%time
results = df.aggregate([column("filename")], [f.sum(column("duration"))]).sort(column("SUM(?table?.duration)").sort(ascending=False)).collect()
len(results)

CPU times: user 4.18 s, sys: 2.57 s, total: 6.76 s
Wall time: 18.5 s


1

In [24]:
%%time
sum([len(results[r].columns[0]) for r in range(len(results))])

CPU times: user 28 µs, sys: 24 µs, total: 52 µs
Wall time: 60.1 µs


1934

In [26]:
%%time
results[0].column(0)[0], results[0].column(1)[0]

CPU times: user 38 µs, sys: 31 µs, total: 69 µs
Wall time: 78.7 µs


(<pyarrow.StringScalar: ''>, <pyarrow.DoubleScalar: 183.01135390461445>)

In [27]:
%%time
results = df.aggregate([column("filename")], [f.sum(column("size"))]).sort(column("SUM(?table?.size)").sort(ascending=False)).collect()
len(results)

CPU times: user 4.01 s, sys: 1.94 s, total: 5.95 s
Wall time: 12.1 s


1

In [28]:
%%time
sum([len(results[r].columns[0]) for r in range(len(results))])

CPU times: user 51 µs, sys: 0 ns, total: 51 µs
Wall time: 58.7 µs


1934

In [29]:
%%time
results[0].column(0)[0], results[0].column(1)[0]

CPU times: user 34 µs, sys: 25 µs, total: 59 µs
Wall time: 65.6 µs


(<pyarrow.StringScalar: '/p/gpfs1/iopp/temp/montage/2854964/5/run/ngc3372.fits'>,
 <pyarrow.Int64Scalar: 38171201556>)

In [30]:
%%time
results = df.aggregate([column("filename"), column("proc_id"), column("io_cat")], [f.sum(column("size"))]).sort(column("SUM(?table?.size)").sort(ascending=False)).collect()
len(results)

CPU times: user 4.97 s, sys: 2.34 s, total: 7.31 s
Wall time: 15 s


2

In [31]:
%%time
sum([len(results[r].columns[0]) for r in range(len(results))])

CPU times: user 46 µs, sys: 29 µs, total: 75 µs
Wall time: 83.7 µs


8486

In [35]:
%%time
results[0].column(0)[0], results[0].column(1)[0], results[0].column(2)[0], results[0].column(3)[0]

CPU times: user 100 µs, sys: 0 ns, total: 100 µs
Wall time: 110 µs


(<pyarrow.StringScalar: '/p/gpfs1/iopp/temp/montage/2854964/5/run/ngc3372.fits'>,
 <pyarrow.Int64Scalar: 5901139297808997828>,
 <pyarrow.Int32Scalar: 1>,
 <pyarrow.Int64Scalar: 37846454032>)

In [36]:
results[0].column(0)[1], results[0].column(1)[1], results[0].column(2)[1], results[0].column(3)[1]

(<pyarrow.StringScalar: '/p/gpfs1/iopp/temp/montage/2854964/25/run/ngc3372.fits'>,
 <pyarrow.Int64Scalar: 9060729382066249156>,
 <pyarrow.Int32Scalar: 1>,
 <pyarrow.Int64Scalar: 10943646736>)

In [37]:
ctx2 = datafusion.SessionContext()
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet/indexed/file_id/*.parquet")
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet/indexed/file_id/*.parquet")
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet/*.parquet")
# df = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet/*.parquet")
ctx2.register_parquet("montage", "/p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet/*.parquet")

ctx2

In [53]:
%%time
sql_df = ctx2.sql("SELECT filename, SUM(duration) FROM montage GROUP BY filename LIMIT 100")
sql_res = sql_df.collect()
sql_res

CPU times: user 4.48 s, sys: 2.4 s, total: 6.88 s
Wall time: 6.82 s


[pyarrow.RecordBatch
 filename: string
 SUM(montage.duration): double,
 pyarrow.RecordBatch
 filename: string
 SUM(montage.duration): double]

In [54]:
len(sql_res)

2

In [57]:
sql_res[0].column(0)[0], sql_res[0].column(1)[0]

(<pyarrow.StringScalar: '/p/gpfs1/iopp/temp/montage/2854964/13/2mass-atlas-000118s-k0880056.fits'>,
 <pyarrow.DoubleScalar: 0.008495999910110186>)

In [59]:
ctx2.register_parquet("hacc", "/p/vast1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet/*.parquet")

In [62]:
%%time
sql_hacc_df = ctx2.sql("SELECT filename, SUM(duration) FROM hacc GROUP BY filename LIMIT 100")
sql_hacc_res = sql_hacc_df.collect()
sql_hacc_res

CPU times: user 335 ms, sys: 514 ms, total: 849 ms
Wall time: 671 ms


[pyarrow.RecordBatch
 filename: string
 SUM(hacc.duration): double,
 pyarrow.RecordBatch
 filename: string
 SUM(hacc.duration): double]

In [63]:
sql_hacc_res[0].column(0)[0], sql_hacc_res[0].column(1)[0]

(<pyarrow.StringScalar: '/p/gpfs1/iopp/temp/hacc_dir//test-Part00000750-of-00001280.data'>,
 <pyarrow.DoubleScalar: 23.446508407592773>)

In [64]:
ctx2.register_parquet("genome", "/p/vast1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet/*.parquet")

In [ ]:
%%time
sql_gen_df = ctx2.sql("SELECT filename, SUM(duration) FROM genome GROUP BY filename LIMIT 100")
sql_gen_res = sql_gen_df.collect()
sql_gen_res

In [3]:
gen_df_0 = ctx.read_parquet("/p/vast1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet/0_0.parquet")
gen_df_0

In [33]:
%%time
gen_res_0 = gen_df_0.aggregate(
    [column("filename"), column("proc_id"), column("io_cat")], 
    [f.sum(column("duration")), f.sum(column("size")), f.count(column("index"))]
).sort(column("COUNT(?table?.index)").sort(ascending=False)).collect()
len(gen_res_0)

CPU times: user 97.4 ms, sys: 40.3 ms, total: 138 ms
Wall time: 180 ms


1

In [34]:
gen_res_0[0].column(0)[0], gen_res_0[0].column(1)[0], gen_res_0[0].column(2)[0], gen_res_0[0].column(3)[0], gen_res_0[0].column(4)[0], gen_res_0[0].column(5)[0]

(<pyarrow.StringScalar: ''>,
 <pyarrow.Int64Scalar: 7523492661920059012>,
 <pyarrow.Int32Scalar: 0>,
 <pyarrow.DoubleScalar: 0.049798899821666964>,
 <pyarrow.Int64Scalar: 0>,
 <pyarrow.Int64Scalar: 9160>)

In [37]:
gen_res_0[0].to_pandas()

,filename,proc_id,io_cat,SUM(?table?.duration),SUM(?table?.size),COUNT(?table?.index)
0,,7523492661920059012,0,0.049799,0,9160
1,%p,6865593879370974948,3,0.004954,0,1746
2,%p,6865593827831367396,3,0.003910,0,1740
3,%p,6691484417816315620,3,0.005259,0,1739
4,%p,6691484434996184804,3,0.005689,0,1739
...,...,...,...,...,...,...
3341,/p/gpfs1/iopp/temp/1000-genome-haridev/scratch...,7523492661920059012,3,0.002793,0,1
3342,/p/gpfs1/iopp/temp/1000-genome-haridev/scratch...,7523492661920059012,3,0.002850,0,1
3343,/p/gpfs1/iopp/temp/1000-genome-haridev/scratch...,7523492661920059012,3,0.002582,0,1
3344,/p/gpfs1/iopp/temp/1000-genome-haridev/scratch...,7523492661920059012,3,0.002947,0,1


In [7]:
cm1_dir = "/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet"

cm1_ddf = dd.read_parquet(f"{cm1_dir}/*.parquet")

cm1_ddf

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
npartitions=1,,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,int32,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [1]:
import polars as pl

# cm1_pl_df = pl.read_parquet(f"{cm1_dir}/*.parquet")

# cm1_pl_df

In [ ]:
%%time
cm1_pl_agg_df = cm1_pl_df \
    .groupby(["filename", "proc_id", "io_cat"]) \
    .agg([pl.sum("duration"), pl.sum("size")]) \
    .sort("duration", reverse=True)
cm1_pl_agg_df


In [ ]:
cm1_pl_agg_df[0]['filename']

In [3]:
gen_dir = "/p/vast1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"
# gen_pl_df = pl.read_parquet(f"{gen_dir}/*.parquet", n_rows=10)

In [20]:
q1 = (
    pl.scan_parquet(f"{gen_dir}/*.parquet")
    .groupby(["file_id", "proc_id", "io_cat"])
    .agg([pl.sum("duration"), pl.sum("size")])
    .sort("duration", reverse=True)
)
# q1

In [21]:
%%time
q1.collect()